# Importing Dependcies

In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 27 kB/s 


# Plug in your dataset




In [3]:
from google.colab import drive
drive.mount("MyDrive")

Mounted at MyDrive


In [4]:
!ls

MyDrive  sample_data


In [5]:
os.chdir('MyDrive/MyDrive/Colab Notebooks/AutoClass')

# Dataset Prep phase

In [82]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

#To check the validity of your dataset
#sklearn.model_selection.permutation_test_score()

def get_dataset(dataset):

    train_test_vars = [
        0.25,
        0
    ]
    Train_test_vars = {
        "test_size": 0.25,
        "random_split": 0,
        "NotInt_Columns": []
    }

    # Remember to put your dependent variable in the last column of your csv file
    dataset = pd.read_csv("{}".format(dataset))
    X = dataset.iloc[:, :-1].values
    Y = dataset.iloc[:, -1].values

    
    #imputer = SimpleImputer(missing_values = np.nan, strategy = "mean")
    #Index doesn't include end value
    #imputer.fit(X[:, 1:-1])
    #X[:, 1:-1] = imputer.transform(X[:, 1:-1])

    #Add this Encoder if you have non-int values

    #try:
     #ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), Train_test_vars["NotInt_Columns"])], remainder='passthrough')
     #try:
       #X = np.array(ct.fit_transform(X))
     #except:
       
    #except TypeError:
       #return "Couldn't encode your not int dependent variable"


    try:
     le = LabelEncoder()
     Y = le.fit_transform(Y)
    except TypeError:
      print("Couldn't encode your independent variable")

    
    #test_size, random_state,  = train_test_vars.values(), train_test_vars.values()

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = Train_test_vars["test_size"], random_state = Train_test_vars["random_split"])
    
    return X_train, X_test, Y_train, Y_test

#def kernel_SVM(X_train, X_test, Y_train, Y_test):
    
def feature_scaling(X_train, X_test):

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test

def final_scoring(X_train, Y_train):

    #cv var is variable
    accuracies = cross_val_score(estimator = classifier, X = X_train, Y = Y_train, cv = 10)
    mean_accuracy, std_accuracy = accuracies.mean()*100, accuracies.std()*100
    
    return mean_accuracy, std_accuracy


# Feature Encoders

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(X)
encoder.fit_transform(X)

# Dimensionality Reduction Functions

In [ ]:
def dim_reduce(X_train, X_test):
  from sklearn.decomposition import PCA
  pca = PCA(n_components = 2)
  X_train = pca.fit_transform(X_train)
  X_test = pca.transform(X_test)

  return X_train, X_test

In [ ]:
def dim_reduce(X_train, X_test):
  from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
  lda = LDA(n_components = 2)
  X_train = lda.fit_transform(X_train, Y_train)
  X_test = lda.transform(X_test)

  return X_train, X_test

In [ ]:
def dim_reduce(X_train, X_test):
  from sklearn.decomposition import KernelPCA  
  kpca = KernelPCA(n_components = 2, kernel = 'rbf')
  X_train = kpca.fit_transform(X_train)
  X_test = kpca.transform(X_test)

  return X_train, X_test

# Models

In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
#from Format import final_scoring_system

Classifier_dict = {
    1: {"name": SVC, "fine_tuning": [
        {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
        {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}], "Scale": 1},
    2: {"name":KNeighborsClassifier , "fine_tuning": [{
        "n_neighbors": [5, 10, 15, 20], 'metric': ['minkowski'], "p": [2]}], "Scale": 1},
    3: {"name": CatBoostClassifier, "fine_tuning": {}, "Scale": 0},
    4: {"name": GaussianNB, "fine_tuning": {}, "Scale": 1},
    5: {"name":LogisticRegression, "fine_tuning": [{
        'random_state': [0,1]}], "Scale": 1},
}

Clustering_dict = {
    1: {"name": "k_means"}
}

#Adding precision & roc accuracy score 
#cv to change
#Kernel PCA Implementation
def fine_tune_scoring(key, X_train, Y_train):

    from sklearn.model_selection import GridSearchCV

    parameters = Classifier_dict[key]["fine_tuning"] 
    grid_search = GridSearchCV(estimator = Classifier_dict[key]["name"](),
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1,
                           )

    grid_search.fit(X_train, Y_train)
    best_accuracy = grid_search.best_score_
    best_parameters = grid_search.best_params_

    return best_accuracy*100, best_parameters


def cm_scoring(classifier, X_test, Y_test):

    from sklearn.metrics import confusion_matrix, accuracy_score

    Y_pred = classifier.predict(X_test)
    cm = confusion_matrix(Y_test, Y_pred)
    test_cm_score = accuracy_score(Y_test, Y_pred)   

    return cm, test_cm_score


# Run to check

In [54]:
def main(dataset):
    print("Send me your CSV please")
    best_fine_tune_score = 0
    best_cm_score = 0

    X_train, X_test, Y_train, Y_test = get_dataset(dataset)
    #X_train, X_test = dim_reduce(X_train, X_test)


    for key in Classifier_dict:

        classifier = Classifier_dict[key]["name"]()

        if Classifier_dict[key]["Scale"] == 1:
            X_train, X_test = feature_scaling(X_train, X_test)

        if Classifier_dict[key]["Scale"] == 1:
            fine_tuned_score, best_params = fine_tune_scoring(key, X_train, Y_train)
            classifier = classifier.set_params(**best_params)
            classifier.fit(X_train, Y_train)
            cm, test_cm_score = cm_scoring(classifier, X_test, Y_test)
        else:
            classifier.fit(X_train, Y_train)
            cm, test_cm_score = cm_scoring(classifier, X_test, Y_test)
            
        if best_fine_tune_score < fine_tuned_score: 
            best_fine_tune_score = fine_tuned_score
            train_classifier_name = "{}".format(classifier)
            best_train_parameters = best_params
            
        
        if best_cm_score < test_cm_score:
            best_cm_score = test_cm_score
            best_cm = cm
            test_classifier_name = "{}".format(classifier)
            best_test_parameters = best_params
        
        #If best_fine_tune_score == fine_tuned_score
        #if best_cm_score == test_cm_score
          
        cm = 0
        test_cm_score = 0
        fine_tuned_score = 0
        best_params = 0


    if (train_classifier_name == test_classifier_name) & (best_cm_score == best_fine_tune_score):
        print("Best test classifier: {} {} \n Parameters: {} Confusion Matrix {}".format(train_classifier_name, best_cm_score, best_test_parameters, best_cm))
    else: 
        print("Best train classifier: {} {}% \n Parameters: {}".format(train_classifier_name, best_fine_tune_score, best_train_parameters))
        print("Best test classifier: {} {} \n Parameters: {} Confusion Matrix {}".format(test_classifier_name, best_cm_score, best_test_parameters, best_cm))




In [83]:
main('Social_Network_Ads.csv')

Send me your CSV please
Learning rate set to 0.006161
0:	learn: 0.6872926	total: 485us	remaining: 485ms
1:	learn: 0.6799471	total: 1.39ms	remaining: 695ms
2:	learn: 0.6721264	total: 2.21ms	remaining: 735ms
3:	learn: 0.6652568	total: 5.77ms	remaining: 1.44s
4:	learn: 0.6586874	total: 6.41ms	remaining: 1.28s
5:	learn: 0.6522441	total: 7.16ms	remaining: 1.19s
6:	learn: 0.6454735	total: 9.33ms	remaining: 1.32s
7:	learn: 0.6390894	total: 9.93ms	remaining: 1.23s
8:	learn: 0.6318089	total: 10.5ms	remaining: 1.16s
9:	learn: 0.6250851	total: 11.1ms	remaining: 1.1s
10:	learn: 0.6185944	total: 11.7ms	remaining: 1.05s
11:	learn: 0.6135007	total: 12.2ms	remaining: 1.01s
12:	learn: 0.6077418	total: 12.8ms	remaining: 972ms
13:	learn: 0.6017832	total: 13.4ms	remaining: 941ms
14:	learn: 0.5956528	total: 13.9ms	remaining: 915ms
15:	learn: 0.5906843	total: 14.5ms	remaining: 893ms
16:	learn: 0.5861526	total: 15.1ms	remaining: 874ms
17:	learn: 0.5807810	total: 15.7ms	remaining: 856ms
18:	learn: 0.5758800	t